In [8]:
from traceroute import (
    load_samples,
    RouteResponse,
    RouterResponse,
    NoResponse,
)
from stats import number_of_negative_ttls, router_response_count

In [7]:
destinations = ["melbourne", "osaka", "oxford", "stanford"]


[0.5714285714285714, 0.5384615384615384, 0.38461538461538464, 0.38461538461538464, 0.42857142857142855]
